## Exercise 1: Exceptional Olympians

Scrape data from [this wikipedia site](https://en.wikipedia.org/wiki/List_of_multiple_Olympic_medalists) about exceptional Olympic medalists. 

1. Download the html using urllib. 
2. Parse this html with BeautifulSoup.
3. Extract the html that corresponds to the big table from the soup.
4. Parse the table into a pandas dataframe. Hint: both the "No." and the "Total." column use row-spans which are tricky to parse, both with a pandas reader and manually. For the purpose of this exercise, exclude all rows that are not easy to parse (the first one is Bjørn Dæhlie).
5. Create a table that shows for each country how many gold, silver, bronze, and total medals it won in that list.

In [2]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

Download the html using urllib. 
Parse this html with BeautifulSoup.

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_multiple_Olympic_medalists"

req = urllib.request.Request(url)
with urllib.request.urlopen(req) as response:
    html = response.read()

print(html)

class_soup = BeautifulSoup(html, 'html.parser')

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of multiple Olympic medalists - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xj0a0wpAAD8AAC05NRkAAADM","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_multiple_Olympic_medalists","wgTitle":"List of multiple Olympic medalists","wgCurRevisionId":937927748,"wgRevisionId":937927748,"wgArticleId":18855244,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with

Extract the html that corresponds to the big table from the soup.

In [4]:
# we can retreive all tables, our desired table is the first one:
table_html = class_soup("table")[0]
table_html

<table class="wikitable sortable">
<tbody><tr>
<th>No.
</th>
<th style="width:7.8em;">Athlete
</th>
<th style="width:8.2em;">Nation
</th>
<th style="width:5.6em;">Sport
</th>
<th>Years
</th>
<th>Games
</th>
<th>Gender
</th>
<th style="background-color:gold; width:3.5em; font-weight:bold;">Gold
</th>
<th style="background-color:silver; width:3.5em; font-weight:bold;">Silver
</th>
<th style="background-color:#cc9966; width:3.5em; font-weight:bold;">Bronze
</th>
<th style="width:3.5em;">Total
</th></tr>
<tr>
<td>1
</td>
<td align="left"><span data-sort-value="Phelps, Michael"><span class="vcard"><span class="fn"><a href="/wiki/Michael_Phelps" title="Michael Phelps">Michael Phelps</a></span></span></span></td>
<td align="left"><img alt="" class="thumbborder" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/22px-Flag_of_the_United_States.svg.png" srcset="//upload.wikimedia.org/wikiped

Parse the table into a pandas dataframe. 

In [5]:
athletes = pd.read_html(str(table_html), header=0)[0]
athletes.head(15)

,No.,Athlete,Nation,Sport,Years,Games,Gender,Gold,Silver,Bronze,Total
0,1,Michael Phelps,United States,Swimming,2004–2016,Summer,M,23,3,2,28
1,2,Larisa Latynina,Soviet Union,Gymnastics,1956–1964,Summer,F,9,5,4,18
2,3,Marit Bjørgen,Norway,Cross-country skiing,2002–2018,Winter,F,8,4,3,15
3,4,Nikolai Andrianov,Soviet Union,Gymnastics,1972–1980,Summer,M,7,5,3,15
4,5,Ole Einar Bjørndalen,Norway,Biathlon,1998–2014,Winter,M,8,4,1,13
5,6,Boris Shakhlin,Soviet Union,Gymnastics,1956–1964,Summer,M,7,4,2,13
6,7,Edoardo Mangiarotti,Italy,Fencing,1936–1960,Summer,M,6,5,2,13
7,8,Takashi Ono,Japan,Gymnastics,1952–1964,Summer,M,5,4,4,13
8,9,Paavo Nurmi,Finland,Athletics,1920–1928,Summer,M,9,3,0,12
9,10,Birgit Fischer,East Germany Germany,Canoeing,1980–2004,Summer,F,8,4,0,12


For cases where the row is screwed up, the bronze column is NaN, which we can use to filter:

In [6]:
athletes = athletes[pd.notnull(athletes["Bronze"])]

Subset to the relevant columns: 

In [7]:
athletes = athletes[["Nation", "Gold", "Silver", "Bronze"]]
athletes.head(10)

,Nation,Gold,Silver,Bronze
0,United States,23,3,2
1,Soviet Union,9,5,4
2,Norway,8,4,3
3,Soviet Union,7,5,3
4,Norway,8,4,1
5,Soviet Union,7,4,2
6,Italy,6,5,2
7,Japan,5,4,4
8,Finland,9,3,0
9,East Germany Germany,8,4,0


Grouping, summing, calculating the total, and sorting: 

In [8]:
countries = athletes.groupby("Nation").sum()

In [9]:
countries["Total"] = countries["Gold"] + countries["Silver"] + countries["Bronze"]
countries.sort_values("Total", ascending=False)

,Gold,Silver,Bronze,Total
Nation,,,,
United States,143,57,42,242
Soviet Union,53,48,26,127
Norway,44,24,10,78
Italy,26,23,22,71
Japan,36,17,12,65
Germany,24,23,15,62
Hungary,30,15,12,57
Finland,30,13,14,57
Australia,24,22,10,56


## Exercise 2 – APIs

Use the [Open Notify API](http://open-notify.org/Open-Notify-API/People-In-Space/) to find out how many people are in space right now.

In [1]:
import requests 

url = "http://api.open-notify.org/astros.json"

r = requests.get(url)
data = r.json()
data

{'people': [{'craft': 'ISS', 'name': 'Andrew Morgan'},
  {'craft': 'ISS', 'name': 'Oleg Skripochka'},
  {'craft': 'ISS', 'name': 'Jessica Meir'}],
 'message': 'success',
 'number': 3}